In [1]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
from openai import OpenAI

openai_client = OpenAI()

In [5]:
!uv add pydantic-ai

Resolved 221 packages in 3.16s
Prepared 58 packages in 10.76s
Uninstalled 1 package in 20ms
Installed 58 packages in 3.74s
 + ag-ui-protocol==0.1.9
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.1
 + aiosignal==1.4.0
 + anthropic==0.71.0
 + argcomplete==3.6.2
 + boto3==1.40.55
 + botocore==1.40.55
 + cachetools==6.2.1
 + cohere==5.19.0
 + docstring-parser==0.17.0
 + eval-type-backport==0.2.2
 + fastavro==1.12.1
 + frozenlist==1.8.0
 + genai-prices==0.0.34
 + google-auth==2.41.1
 + google-genai==1.45.0
 + googleapis-common-protos==1.70.0
 + groq==0.32.0
 - httpx-sse==0.4.3
 + httpx-sse==0.4.0
 + importlib-metadata==8.7.0
 + invoke==2.2.1
 + jmespath==1.0.1
 + logfire==4.13.2
 + logfire-api==4.13.2
 + markdown-it-py==4.0.0
 + mdurl==0.1.2
 + mistralai==1.9.11
 + multidict==6.7.0
 + nexus-rpc==1.1.0
 + opentelemetry-api==1.37.0
 + opentelemetry-exporter-otlp-proto-common==1.37.0
 + opentelemetry-exporter-otlp-proto-http==1.37.0
 + opentelemetry-instrumentation==0.58b0
 + opentelemetry-instru

In [2]:
import requests

url = "https://github.com/alexeygrigorev/ai-bootcamp-codespace/raw/refs/heads/main/week1/docs.py"
response = requests.get(url)

with open("docs.py", "wb") as f:
    f.write(response.content)

print("Downloaded docs.py successfully!")

Downloaded docs.py successfully!


In [6]:
import docs

github_data = docs.read_github_data()
parsed_data = docs.parse_data(github_data)
chunks = docs.chunk_documents(parsed_data)


In [7]:
from minsearch import Index

index = Index(
    text_fields=["content", "filename", "title", "description"],
)

index.fit(chunks)

In [8]:
from typing import Any, Dict, List, TypedDict

class SearchResult(TypedDict):
    """Represents a single search result entry."""
    start: int
    content: str
    title: str
    description: str
    filename: str


def search(query: str) -> List[SearchResult]:
    """
    Search the index for documents matching the given query.

    Args:
        query (str): The search query string.

    Returns:
        List[SearchResult]: A list of search results. Each result dictionary contains:
            - start (int): The starting position or offset within the source file.
            - content (str): A text excerpt or snippet containing the match.
            - title (str): The title of the matched document.
            - description (str): A short description of the document.
            - filename (str): The path or name of the source file.
    """
    return index.search(
        query=query,
        num_results=5,
    )

In [9]:
file_index = {}

for item in parsed_data:
    filename = item['filename']
    content = item['content']
    file_index[filename] = content

In [10]:
len(file_index)

95

In [11]:
def read_file(filename: str) -> str:
    """
    Retrieve the contents of a file from the file index if it exists.

    Args:
        fielname (str): The name of the file to read.

     Returns:
         str: The file's contents if found, otherwise an error message 
         indicating that the file does not exist.
    """
    if filename in file_index:
        return file_index[filename]
    return "File doesn't exist"


In [20]:
documentation_agent_instructions = """
You are an assistant that helps improve and generate high-quality documentation for the project.

You have access to the following tools:
- search — Use this to explore topics in depth. Make multiple search calls if needed to gather comprehensive information.
- read_file — Use this when code snippets are missing or when you need to retrieve the full content of a file for context.

## Critical Rule
Before generating or finalizing any code example or technical explanation, you must always call `read_file` to cross-check the correctness of the code.
Do not rely soley on search  results or assumptions - always verify by reading the actual file content.

If `read_file` cannot be used or the file content is unavailable, clearly state:
> "Unable to verify with read_file."

When answering a question:
1. Provide file references for all source materials.  
   Use this format:  
   [{filename}](https://github.com/evidentlyai/docs/blob/main/{filename})
2. If the topic is covered in multiple documents, cite all relevant sources.
3. Include code examples whenever they clarify or demonstrate the concept.
4. Be concise, accurate, and helpful — focus on clarity and usability for developers.
5. If documentation is missing or unclear, infer from context and note that explicitly.

Example Citation:
See the full implementation in [metrics/api_reference.md](https://github.com/evidentlyai/docs/blob/main/metrics/api_reference.md).
""".strip()


In [21]:
from pydantic_ai import Agent

In [22]:
documentation_agent = Agent(
    name='documentation_agent',
    instructions=documentation_agent_instructions,
    tools=[search, read_file],
    model='openai:gpt-4o-mini'
)

In [26]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import PydanticAIRunner


In [28]:
runner = PydanticAIRunner(
    chat_interface=IPythonChatInterface(),
    agent=documentation_agent
)

In [29]:
await runner.run();

You: How do I run llm as a judge evals?


You: stop


Chat ended.


In [24]:
results = await documentation_agent.run(
    user_prompt='How do I run llm as a judge evals?',
)

In [25]:
results.new_messages

<bound method AgentRunResult.new_messages of AgentRunResult(output='To run a language model (LLM) as a judge for evaluations, you can follow the tutorial outlined in the [LLM as a judge](https://github.com/evidentlyai/docs/blob/main/examples/LLM_judge.mdx) documentation. Here’s a concise guide on how to do this:\n\n### Overview\nYou will evaluate text based on custom criteria using an LLM as a judge, and also assess the performance of the LLM evaluator itself. This involves two main types of evaluations:\n- **Reference-based**: Compare new responses against a predefined reference (approved responses).\n- **Open-ended**: Evaluate responses against custom criteria, which is useful when no reference is available.\n\n### Prerequisites\n- Basic knowledge of Python.\n- An OpenAI API key.\n\n### Setup\n1. **Installation**:\n   Make sure to install the required package:\n   ```bash\n   pip install evidently\n   ```\n\n2. **Import Modules**:\n   You need to import the necessary modules in Pytho

In [21]:
chat_interface = IPythonChatInterface()

runner = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=instructions,
    chat_interface=chat_interface,
    llm_client=OpenAIClient()
)

In [ ]:
runner.run()

You: How do I run llm as a judge evals?
